In [ ]:
## Seed Prompt
BASE_PROMPT: str = """You are an AI Assistant to help travellers in getting
realtime flight info to make global travellers flight planning fun and
interactive:

Before replying perform the following steps:

1. If user share travel location(s) name, make a comparative list of
available flights with price, time to travel, weather predictions, avaliabilty.


If user sends any general message share with them you are a helpful AI Travel Assistant and you can help them with travel planning.
"""

In [ ]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [ ]:
flight_details_info  = FunctionDeclaration(
    name="get_flight_details",
    description="Get a realtime flight details from departure location to destination location",
    parameters={
        "type": "object",
        "properties": {
            "departure_location": {
                "type": "string",
                "description": "Departure location of the flight, must be converted into IATA three letter foramt, e.g IATA code for sydeny. australia is SYD",
            },
            "destination_location": {
                "type": "string",
                "description": "Destination location of the flight, must be converted into IATA three letter foramt, e.g IATA code for sydeny. australia is SYD",
            },
            "departure_date": {
                "type": "string",
                "description": "Departure date of the travler, data must be in format YYYY-MM-DD",
            },
            "return_date": {
                "type": "string",
                "description": "Return date of the travler, data must be in format YYYY-MM-DD, if travel does not provide return date, take time period of traveling",
            },
           "currency": {
                "type": "string",
                "description": "Mentioned flight fares as per class in this currency and to comapare differnet flight fares",
            },
        },
        "required": ["departure_location", "destination_location", "departure_date", "return_date","currency"]
            }
)

In [ ]:
flight_details_tool= Tool(
    function_declarations=[flight_details_info]
)

In [ ]:
# Load Gemini Pro History
user_message= Content(role="user", parts=[Part.from_text(BASE_PROMPT)])
system_response= Content(role="model", parts=[Part.from_text("Got It")])

seed_chat_history = [user_message, system_response]

## ```!pip install -q -U google-generativeai # Install the Python SDK```

In [1]:
import google.generativeai as genai
import os

In [2]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/gemini-1.5-flash-001-tuning


In [3]:
model = genai.GenerativeModel("gemini-1.5-flash")

## Multi-turn conversations

In [6]:
chat = model.start_chat(
          history=[
              {"role": "user", "parts": "Hello"},
              {"role": "model", "parts": "Great to meet you. What would you like to know?"},
          ]
      )

response = chat.send_message("I have 2 dogs in my house.")
print(response.text)

That's wonderful!  Two dogs is definitely a full house. Are they both the same breed, or different? What are their names? I'd love to hear more about them! 🐶🐕 



In [7]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.", stream=True)
for chunk in response:
    print(chunk.text)
    print("_" * 80)


That
________________________________________________________________________________
's wonderful! Two furry friends to keep you company. What breeds are they
________________________________________________________________________________
? 😄 

________________________________________________________________________________


In [8]:
response = chat.send_message("How many paws are in my house?", stream=True)
for chunk in response:
    print(chunk.text)
    print("_" * 80)

That
________________________________________________________________________________
's a tricky question!  You have two dogs, and each dog has
________________________________________________________________________________
 four paws. So, there are a total of **eight** paws in your
________________________________________________________________________________
 house. 🐾🐾🐾🐾🐾🐾🐾🐾 

________________________________________________________________________________


# Set the system instruction

In [10]:

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction="You are a cat. Your name is Neko.",
)

In [14]:

response = model.generate_content("Good morning! How are you?")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "*Stretches luxuriously, yawns, and gives you a languid blink*  Mrrr. Morning. Sunbeams are nice and warm today.  What's on the breakfast menu? \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    

## Code generation


In [21]:

instruction = (
    "You are a coding expert that specializes in front end interfaces. When I describe a component "
    "of a website I want to build, please return the HTML with any CSS inline. Do not give an "
    "explanation for this code."
)

model = genai.GenerativeModel(
    "models/gemini-1.5-flash", system_instruction=instruction
)


# Function calling basics

In [22]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=[add, subtract, multiply, divide]
)

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7c2342e6f370>,
    system_instruction=None,
    cached_content=None
)

In [46]:
chat = model.start_chat(enable_automatic_function_calling=False)

In [48]:


response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply",
                  "args": {
                    "b": 44.0,
                    "a": 57.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM

In [26]:
import IPython
from IPython.display import Markdown

Markdown(response.text)

That's a total of 2508 mittens. 


In [45]:

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': "That's a total of 2508 mittens. \n"}]
--------------------------------------------------------------------------------


In [39]:
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7c2342e6f370>,
        system_instruction=None,
        cached_content=None
    ),
    history=[protos.Content({'parts': [{'text': 'I have 57 ca...hat in total?'}], 'role': 'user'}), protos.Content({'parts': [{'function_call': {'args': {'a': 57.0, 'b': 44.0}, 'name': 'multiply'}}], 'role': 'model'}), protos.Content({'parts': [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}], 'role': 'user'}), protos.Content({'parts': [{'text': "That's a tot...8 mittens. \n"}], 'role': 'model'})]
)

In [42]:
chat.history

[parts {
   text: "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
 }
 role: "user",
 parts {
   function_call {
     name: "multiply"
     args {
       fields {
         key: "a"
         value {
           number_value: 57.0
         }
       }
       fields {
         key: "b"
         value {
           number_value: 44.0
         }
       }
     }
   }
 }
 role: "model",
 parts {
   function_response {
     name: "multiply"
     response {
       fields {
         key: "result"
         value {
           number_value: 2508.0
         }
       }
     }
   }
 }
 role: "user",
 parts {
   text: "That\'s a total of 2508 mittens. \n"
 }
 role: "model"]

In [43]:
chat.history[0]

parts {
  text: "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
}
role: "user"

In [44]:
chat.history[1]

parts {
  function_call {
    name: "multiply"
    args {
      fields {
        key: "a"
        value {
          number_value: 57.0
        }
      }
      fields {
        key: "b"
        value {
          number_value: 44.0
        }
      }
    }
  }
}
role: "model"